In [ ]:
# Dataset de ejemplo: Aprobación de préstamos bancarios
import pandas as pd
import numpy as np
from collections import Counter
import math

# Crear dataset sintético para clasificación
datos_prestamos = [
    {'edad': 25, 'ingresos': 30000, 'historial_credito': 'Bueno', 'empleo': 'Estable', 'aprobado': 'Si'},
    {'edad': 35, 'ingresos': 50000, 'historial_credito': 'Excelente', 'empleo': 'Estable', 'aprobado': 'Si'},
    {'edad': 22, 'ingresos': 20000, 'historial_credito': 'Malo', 'empleo': 'Temporal', 'aprobado': 'No'},
    {'edad': 45, 'ingresos': 80000, 'historial_credito': 'Bueno', 'empleo': 'Estable', 'aprobado': 'Si'},
    {'edad': 28, 'ingresos': 35000, 'historial_credito': 'Regular', 'empleo': 'Estable', 'aprobado': 'No'},
    {'edad': 55, 'ingresos': 90000, 'historial_credito': 'Excelente', 'empleo': 'Estable', 'aprobado': 'Si'},
    {'edad': 19, 'ingresos': 15000, 'historial_credito': 'Malo', 'empleo': 'Temporal', 'aprobado': 'No'},
    {'edad': 30, 'ingresos': 45000, 'historial_credito': 'Bueno', 'empleo': 'Estable', 'aprobado': 'Si'},
    {'edad': 40, 'ingresos': 25000, 'historial_credito': 'Regular', 'empleo': 'Temporal', 'aprobado': 'No'},
    {'edad': 32, 'ingresos': 60000, 'historial_credito': 'Excelente', 'empleo': 'Estable', 'aprobado': 'Si'},
    {'edad': 26, 'ingresos': 22000, 'historial_credito': 'Malo', 'empleo': 'Temporal', 'aprobado': 'No'},
    {'edad': 50, 'ingresos': 75000, 'historial_credito': 'Bueno', 'empleo': 'Estable', 'aprobado': 'Si'}
]

# Convertir a DataFrame para facilitar manipulación
df = pd.DataFrame(datos_prestamos)

print("=== DATASET DE PRÉSTAMOS BANCARIOS ===")
print(df)

print(f"\nTotal de instancias: {len(df)}")
print(f"Variables predictoras: {list(df.columns[:-1])}")
print(f"Variable objetivo: {df.columns[-1]}")

# Análisis de distribución de clases
print(f"\nDistribución de clases:")
print(df['aprobado'].value_counts())

# Estadísticas descriptivas básicas
print(f"\nEstadísticas básicas:")
print(f"Edad promedio: {df['edad'].mean():.1f} años")
print(f"Ingresos promedio: ${df['ingresos'].mean():,.0f}")
print(f"Historial de crédito más común: {df['historial_credito'].mode()[0]}")
print(f"Tipo de empleo más común: {df['empleo'].mode()[0]}")

# Análisis por clase
print(f"\n=== ANÁLISIS POR CLASE ===")
for clase in df['aprobado'].unique():
    subset = df[df['aprobado'] == clase]
    print(f"\nClase '{clase}' ({len(subset)} instancias):")
    print(f"  Edad promedio: {subset['edad'].mean():.1f}")
    print(f"  Ingresos promedio: ${subset['ingresos'].mean():,.0f}")
    print(f"  Historial más común: {subset['historial_credito'].mode()[0] if len(subset) > 0 else 'N/A'}")
    print(f"  Empleo más común: {subset['empleo'].mode()[0] if len(subset) > 0 else 'N/A'}")


In [ ]:
# Implementación de Árbol de Decisión desde cero

class NodoArbol:
    def __init__(self):
        self.atributo = None        # Atributo para división
        self.valor_division = None  # Valor umbral para división
        self.ramas = {}            # Diccionario de ramas
        self.clase_predicha = None # Clase predicha (para hojas)
        self.es_hoja = False       # Si es nodo hoja
        self.impureza = 0          # Medida de impureza del nodo

def calcular_entropia(datos, clase_objetivo):
    """Calcula la entropía de un conjunto de datos"""
    if len(datos) == 0:
        return 0
    
    # Contar clases
    clases = [fila[clase_objetivo] for fila in datos]
    contador_clases = Counter(clases)
    total = len(datos)
    
    entropia = 0
    for clase, count in contador_clases.items():
        proporcion = count / total
        if proporcion > 0:  # Evitar log(0)
            entropia -= proporcion * math.log2(proporcion)
    
    return entropia

def calcular_gini(datos, clase_objetivo):
    """Calcula el índice Gini de un conjunto de datos"""
    if len(datos) == 0:
        return 0
    
    clases = [fila[clase_objetivo] for fila in datos]
    contador_clases = Counter(clases)
    total = len(datos)
    
    gini = 1
    for clase, count in contador_clases.items():
        proporcion = count / total
        gini -= proporcion ** 2
    
    return gini

def clase_mayoritaria(datos, clase_objetivo):
    """Encuentra la clase más frecuente en los datos"""
    clases = [fila[clase_objetivo] for fila in datos]
    contador_clases = Counter(clases)
    return contador_clases.most_common(1)[0][0]

# Ejemplos de cálculo de impureza
print("=== CÁLCULO DE MEDIDAS DE IMPUREZA ===")

# Convertir DataFrame a lista de diccionarios para facilitar cálculos
datos_lista = df.to_dict('records')

# Calcular entropía del conjunto completo
entropia_total = calcular_entropia(datos_lista, 'aprobado')
gini_total = calcular_gini(datos_lista, 'aprobado')

print(f"Dataset completo:")
print(f"  Entropía: {entropia_total:.3f}")
print(f"  Índice Gini: {gini_total:.3f}")

# Ejemplo de división por historial de crédito
print(f"\nEjemplo de división por 'historial_credito':")

historiales = df['historial_credito'].unique()
entropia_ponderada = 0
gini_ponderado = 0

for historial in historiales:
    subset = [fila for fila in datos_lista if fila['historial_credito'] == historial]
    proporcion = len(subset) / len(datos_lista)
    entropia_subset = calcular_entropia(subset, 'aprobado')
    gini_subset = calcular_gini(subset, 'aprobado')
    
    print(f"  {historial}: {len(subset)} instancias")
    print(f"    Entropía: {entropia_subset:.3f}")
    print(f"    Gini: {gini_subset:.3f}")
    
    entropia_ponderada += proporcion * entropia_subset
    gini_ponderado += proporcion * gini_subset

# Calcular ganancias de información
ganancia_entropia = entropia_total - entropia_ponderada
ganancia_gini = gini_total - gini_ponderado

print(f"\nGanancias tras división por 'historial_credito':")
print(f"  Ganancia de Información (Entropía): {ganancia_entropia:.3f}")
print(f"  Ganancia Gini: {ganancia_gini:.3f}")


In [ ]:
# Algoritmo completo de construcción de árbol de decisión

def encontrar_mejor_division(datos, atributos, clase_objetivo, criterio='entropia'):
    """Encuentra el mejor atributo y valor para dividir los datos"""
    mejor_ganancia = -1
    mejor_atributo = None
    mejor_valor = None
    mejor_subconjuntos = None
    
    # Calcular impureza inicial
    if criterio == 'entropia':
        impureza_inicial = calcular_entropia(datos, clase_objetivo)
    else:
        impureza_inicial = calcular_gini(datos, clase_objetivo)
    
    for atributo in atributos:
        if atributo == clase_objetivo:
            continue
            
        # Obtener valores únicos del atributo
        valores = list(set([fila[atributo] for fila in datos]))
        
        # Para atributos categóricos
        if isinstance(valores[0], str):
            subconjuntos = {}
            for valor in valores:
                subconjuntos[valor] = [fila for fila in datos if fila[atributo] == valor]
            
            # Calcular impureza ponderada
            impureza_ponderada = 0
            for valor, subset in subconjuntos.items():
                proporcion = len(subset) / len(datos)
                if criterio == 'entropia':
                    impureza_ponderada += proporcion * calcular_entropia(subset, clase_objetivo)
                else:
                    impureza_ponderada += proporcion * calcular_gini(subset, clase_objetivo)
            
            ganancia = impureza_inicial - impureza_ponderada
            
            if ganancia > mejor_ganancia:
                mejor_ganancia = ganancia
                mejor_atributo = atributo
                mejor_valor = 'categorico'
                mejor_subconjuntos = subconjuntos
        
        # Para atributos numéricos
        else:
            valores_ordenados = sorted(valores)
            for i in range(len(valores_ordenados) - 1):
                umbral = (valores_ordenados[i] + valores_ordenados[i + 1]) / 2
                
                izquierda = [fila for fila in datos if fila[atributo] <= umbral]
                derecha = [fila for fila in datos if fila[atributo] > umbral]
                
                if len(izquierda) == 0 or len(derecha) == 0:
                    continue
                
                # Calcular impureza ponderada
                prop_izq = len(izquierda) / len(datos)
                prop_der = len(derecha) / len(datos)
                
                if criterio == 'entropia':
                    impureza_ponderada = (prop_izq * calcular_entropia(izquierda, clase_objetivo) +
                                        prop_der * calcular_entropia(derecha, clase_objetivo))
                else:
                    impureza_ponderada = (prop_izq * calcular_gini(izquierda, clase_objetivo) +
                                        prop_der * calcular_gini(derecha, clase_objetivo))
                
                ganancia = impureza_inicial - impureza_ponderada
                
                if ganancia > mejor_ganancia:
                    mejor_ganancia = ganancia
                    mejor_atributo = atributo
                    mejor_valor = umbral
                    mejor_subconjuntos = {'<=': izquierda, '>': derecha}
    
    return mejor_atributo, mejor_valor, mejor_ganancia, mejor_subconjuntos

def construir_arbol(datos, atributos, clase_objetivo, profundidad=0, max_profundidad=5, min_muestras=2):
    """Construye el árbol de decisión recursivamente"""
    nodo = NodoArbol()
    
    # Criterios de parada
    clases = [fila[clase_objetivo] for fila in datos]
    
    # Si todas las instancias tienen la misma clase
    if len(set(clases)) == 1:
        nodo.es_hoja = True
        nodo.clase_predicha = clases[0]
        return nodo
    
    # Si no hay más atributos o se alcanzó profundidad máxima o muy pocas muestras
    if (len(atributos) <= 1 or profundidad >= max_profundidad or len(datos) < min_muestras):
        nodo.es_hoja = True
        nodo.clase_predicha = clase_mayoritaria(datos, clase_objetivo)
        return nodo
    
    # Encontrar la mejor división
    mejor_atributo, mejor_valor, ganancia, subconjuntos = encontrar_mejor_division(
        datos, atributos, clase_objetivo)
    
    # Si no hay ganancia, crear hoja
    if ganancia <= 0:
        nodo.es_hoja = True
        nodo.clase_predicha = clase_mayoritaria(datos, clase_objetivo)
        return nodo
    
    # Crear nodo interno
    nodo.atributo = mejor_atributo
    nodo.valor_division = mejor_valor
    
    # Crear ramas recursivamente
    for valor, subset in subconjuntos.items():
        if len(subset) > 0:
            nodo.ramas[valor] = construir_arbol(
                subset, atributos, clase_objetivo, 
                profundidad + 1, max_profundidad, min_muestras)
    
    return nodo

# Construir el árbol de decisión
print("\n=== CONSTRUCCIÓN DEL ÁRBOL DE DECISIÓN ===")

atributos = list(df.columns)
arbol = construir_arbol(datos_lista, atributos, 'aprobado', max_profundidad=3, min_muestras=2)

def imprimir_arbol(nodo, nivel=0, rama="raiz"):
    """Imprime la estructura del árbol"""
    indentacion = "  " * nivel
    
    if nodo.es_hoja:
        print(f"{indentacion}└─ {rama}: PREDICCIÓN = {nodo.clase_predicha}")
    else:
        print(f"{indentacion}└─ {rama}: {nodo.atributo} = ?")
        for valor, subnodo in nodo.ramas.items():
            if isinstance(nodo.valor_division, (int, float)):
                if valor == '<=':
                    etiqueta = f"<= {nodo.valor_division}"
                else:
                    etiqueta = f"> {nodo.valor_division}"
            else:
                etiqueta = valor
            imprimir_arbol(subnodo, nivel + 1, etiqueta)

print("Estructura del árbol construido:")
imprimir_arbol(arbol)


In [ ]:
# Función de predicción para el árbol de decisión

def predecir(arbol, instancia):
    """Predice la clase de una nueva instancia usando el árbol"""
    nodo_actual = arbol
    
    while not nodo_actual.es_hoja:
        atributo = nodo_actual.atributo
        valor_instancia = instancia[atributo]
        
        # Para divisiones categóricas
        if nodo_actual.valor_division == 'categorico':
            if valor_instancia in nodo_actual.ramas:
                nodo_actual = nodo_actual.ramas[valor_instancia]
            else:
                # Si el valor no se vio en entrenamiento, usar clase mayoritaria
                return clase_mayoritaria([instancia], list(instancia.keys())[-1])
        
        # Para divisiones numéricas
        else:
            if valor_instancia <= nodo_actual.valor_division:
                nodo_actual = nodo_actual.ramas['<=']
            else:
                nodo_actual = nodo_actual.ramas['>']
    
    return nodo_actual.clase_predicha

# Probar predicciones con instancias del conjunto de entrenamiento
print("\n=== PREDICCIONES DEL ÁRBOL ===")

predicciones_correctas = 0
for i, instancia in enumerate(datos_lista):
    prediccion = predecir(arbol, instancia)
    real = instancia['aprobado']
    es_correcta = prediccion == real
    
    if es_correcta:
        predicciones_correctas += 1
    
    print(f"Instancia {i+1}: Predicción={prediccion}, Real={real}, ✓" if es_correcta 
          else f"Instancia {i+1}: Predicción={prediccion}, Real={real}, ✗")

precision = predicciones_correctas / len(datos_lista)
print(f"\nPrecisión en conjunto de entrenamiento: {precision:.1%}")

# Probar con nuevas instancias
print("\n=== PREDICCIONES PARA NUEVAS INSTANCIAS ===")

nuevas_instancias = [
    {'edad': 27, 'ingresos': 42000, 'historial_credito': 'Bueno', 'empleo': 'Estable'},
    {'edad': 23, 'ingresos': 18000, 'historial_credito': 'Malo', 'empleo': 'Temporal'},
    {'edad': 42, 'ingresos': 65000, 'historial_credito': 'Excelente', 'empleo': 'Estable'},
    {'edad': 31, 'ingresos': 28000, 'historial_credito': 'Regular', 'empleo': 'Temporal'}
]

for i, instancia in enumerate(nuevas_instancias, 1):
    prediccion = predecir(arbol, instancia)
    print(f"Nueva instancia {i}:")
    print(f"  Perfil: Edad={instancia['edad']}, Ingresos=${instancia['ingresos']:,}, "
          f"Historial={instancia['historial_credito']}, Empleo={instancia['empleo']}")
    print(f"  Predicción: {prediccion}")
    print()


In [ ]:
# Implementación de Clasificador Basado en Reglas

class Regla:
    def __init__(self):
        self.condiciones = []    # Lista de condiciones (atributo, operador, valor)
        self.clase_predicha = None
        self.confianza = 0
        self.soporte = 0
        self.instancias_cubiertas = 0

    def agregar_condicion(self, atributo, operador, valor):
        """Agrega una condición a la regla"""
        self.condiciones.append((atributo, operador, valor))
    
    def evaluar(self, instancia):
        """Evalúa si la instancia satisface todas las condiciones de la regla"""
        for atributo, operador, valor in self.condiciones:
            valor_instancia = instancia[atributo]
            
            if operador == '=':
                if valor_instancia != valor:
                    return False
            elif operador == '>':
                if valor_instancia <= valor:
                    return False
            elif operador == '<=':
                if valor_instancia > valor:
                    return False
            elif operador == '>=':
                if valor_instancia < valor:
                    return False
            elif operador == '<':
                if valor_instancia >= valor:
                    return False
        
        return True
    
    def __str__(self):
        condiciones_str = []
        for atributo, operador, valor in self.condiciones:
            if isinstance(valor, str):
                condiciones_str.append(f"{atributo} {operador} '{valor}'")
            else:
                condiciones_str.append(f"{atributo} {operador} {valor}")
        
        return f"SI ({' Y '.join(condiciones_str)}) ENTONCES aprobado = '{self.clase_predicha}'"

def generar_reglas_simples(datos, clase_objetivo):
    """Genera reglas simples basadas en análisis de los datos"""
    reglas = []
    
    # Regla 1: Ingresos altos y buen historial
    regla1 = Regla()
    regla1.agregar_condicion('ingresos', '>', 45000)
    regla1.agregar_condicion('historial_credito', '=', 'Bueno')
    regla1.clase_predicha = 'Si'
    
    # Regla 2: Ingresos muy altos
    regla2 = Regla()
    regla2.agregar_condicion('ingresos', '>', 70000)
    regla2.clase_predicha = 'Si'
    
    # Regla 3: Historial excelente
    regla3 = Regla()
    regla3.agregar_condicion('historial_credito', '=', 'Excelente')
    regla3.clase_predicha = 'Si'
    
    # Regla 4: Joven con empleo temporal
    regla4 = Regla()
    regla4.agregar_condicion('edad', '<', 25)
    regla4.agregar_condicion('empleo', '=', 'Temporal')
    regla4.clase_predicha = 'No'
    
    # Regla 5: Historial malo
    regla5 = Regla()
    regla5.agregar_condicion('historial_credito', '=', 'Malo')
    regla5.clase_predicha = 'No'
    
    # Regla 6: Ingresos bajos
    regla6 = Regla()
    regla6.agregar_condicion('ingresos', '<=', 25000)
    regla6.clase_predicha = 'No'
    
    reglas = [regla1, regla2, regla3, regla4, regla5, regla6]
    
    # Calcular métricas para cada regla
    for regla in reglas:
        instancias_que_aplican = 0
        predicciones_correctas = 0
        
        for instancia in datos:
            if regla.evaluar(instancia):
                instancias_que_aplican += 1
                if instancia[clase_objetivo] == regla.clase_predicha:
                    predicciones_correctas += 1
        
        regla.instancias_cubiertas = instancias_que_aplican
        regla.soporte = instancias_que_aplican / len(datos)
        regla.confianza = predicciones_correctas / max(instancias_que_aplican, 1)
    
    return reglas

class ClasificadorReglas:
    def __init__(self, estrategia_conflicto='orden'):
        self.reglas = []
        self.estrategia_conflicto = estrategia_conflicto
        self.clase_por_defecto = None
    
    def entrenar(self, datos, clase_objetivo):
        """Entrena el clasificador generando reglas"""
        self.reglas = generar_reglas_simples(datos, clase_objetivo)
        
        # Determinar clase por defecto (más frecuente)
        clases = [instancia[clase_objetivo] for instancia in datos]
        self.clase_por_defecto = Counter(clases).most_common(1)[0][0]
        
        # Ordenar reglas por confianza descendente
        self.reglas.sort(key=lambda r: r.confianza, reverse=True)
    
    def predecir(self, instancia):
        """Predice la clase de una instancia"""
        reglas_aplicables = []
        
        for regla in self.reglas:
            if regla.evaluar(instancia):
                reglas_aplicables.append(regla)
        
        if not reglas_aplicables:
            return self.clase_por_defecto
        
        # Estrategias de resolución de conflictos
        if self.estrategia_conflicto == 'orden':
            return reglas_aplicables[0].clase_predicha
        
        elif self.estrategia_conflicto == 'confianza':
            mejor_regla = max(reglas_aplicables, key=lambda r: r.confianza)
            return mejor_regla.clase_predicha
        
        elif self.estrategia_conflicto == 'voto':
            votos = Counter([regla.clase_predicha for regla in reglas_aplicables])
            return votos.most_common(1)[0][0]
        
        return self.clase_por_defecto

# Probar el clasificador basado en reglas
print("=== CLASIFICADOR BASADO EN REGLAS ===")

clasificador_reglas = ClasificadorReglas(estrategia_conflicto='confianza')
clasificador_reglas.entrenar(datos_lista, 'aprobado')

print("Reglas generadas:")
for i, regla in enumerate(clasificador_reglas.reglas, 1):
    print(f"{i}. {regla}")
    print(f"   Soporte: {regla.soporte:.3f}, Confianza: {regla.confianza:.3f}, "
          f"Instancias: {regla.instancias_cubiertas}")
    print()


In [ ]:
# Evaluación y comparación de modelos

print("=== EVALUACIÓN DE PREDICCIONES ===")

# Predicciones del clasificador de reglas
predicciones_reglas_correctas = 0
print("Clasificador de Reglas:")
for i, instancia in enumerate(datos_lista):
    prediccion = clasificador_reglas.predecir(instancia)
    real = instancia['aprobado']
    es_correcta = prediccion == real
    
    if es_correcta:
        predicciones_reglas_correctas += 1
    
    print(f"Instancia {i+1}: Predicción={prediccion}, Real={real}, ✓" if es_correcta 
          else f"Instancia {i+1}: Predicción={prediccion}, Real={real}, ✗")

precision_reglas = predicciones_reglas_correctas / len(datos_lista)

print(f"\n=== COMPARACIÓN DE MÉTODOS ===")
print(f"Árbol de Decisión - Precisión: {precision:.1%}")
print(f"Clasificador de Reglas - Precisión: {precision_reglas:.1%}")

# Análisis detallado de predicciones en nuevas instancias
print(f"\n=== COMPARACIÓN EN NUEVAS INSTANCIAS ===")

for i, instancia in enumerate(nuevas_instancias, 1):
    pred_arbol = predecir(arbol, instancia)
    pred_reglas = clasificador_reglas.predecir(instancia)
    
    print(f"Nueva instancia {i}:")
    print(f"  Perfil: Edad={instancia['edad']}, Ingresos=${instancia['ingresos']:,}")
    print(f"  Historial={instancia['historial_credito']}, Empleo={instancia['empleo']}")
    print(f"  Árbol de Decisión: {pred_arbol}")
    print(f"  Clasificador de Reglas: {pred_reglas}")
    
    if pred_arbol == pred_reglas:
        print(f"  ✓ Ambos métodos coinciden")
    else:
        print(f"  ✗ Métodos difieren")
    print()

# Análisis de interpretabilidad
print(f"=== ANÁLISIS DE INTERPRETABILIDAD ===")

print("Ventajas del Árbol de Decisión:")
print("- Estructura jerárquica clara")
print("- Decisiones paso a paso")
print("- Identificación automática de atributos importantes")

print("\nVentajas del Clasificador de Reglas:")
print("- Reglas independientes y modulares")
print("- Fácil modificación de reglas específicas")
print("- Incorporación de conocimiento experto")
print("- Flexibilidad en estrategias de conflicto")

# Análisis de qué reglas se aplicaron a cada instancia
print(f"\n=== ANÁLISIS DE REGLAS APLICADAS ===")

for i, instancia in enumerate(nuevas_instancias[:2], 1):  # Solo primeras 2 para brevedad
    print(f"Nueva instancia {i}:")
    reglas_aplicables = [regla for regla in clasificador_reglas.reglas if regla.evaluar(instancia)]
    
    if reglas_aplicables:
        print(f"  Reglas que aplican:")
        for j, regla in enumerate(reglas_aplicables, 1):
            print(f"    {j}. {regla}")
            print(f"       Confianza: {regla.confianza:.3f}")
    else:
        print(f"  Ninguna regla aplica, usando clase por defecto: {clasificador_reglas.clase_por_defecto}")
    print()
